# Load Hs Features

In [38]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from joblib import Parallel, delayed
from tqdm import tqdm


In [40]:
# Set working directory
#os.chdir("/to/CLEARER_directory/")
EsInfo = pd.read_csv("Class_labels/Hs.csv", sep=",", header=0)
EsInfo = EsInfo[EsInfo['Essential CEG'].notna()] # remove rows with NaN in 'Essential CEG'
print(EsInfo.head())
print(EsInfo['Essential CEG'].value_counts())

# Generate class labels suitable for Python
EsInfo['Essential CEG'] = EsInfo['Essential CEG'].astype('category').cat.codes
EsInfo.set_index('Gene', inplace=True)
print(EsInfo['Essential CEG'].value_counts())

# Load combined features
Data = pd.read_csv("Features/Hs_features.csv.gz", sep=",", header=0, compression='gzip')
Data.set_index('genes', inplace=True)

# reduce to sahred index
idxcommon = np.intersect1d(EsInfo.index.values, Data.index.values)
Data = Data.loc[idxcommon]
EsInfo = EsInfo.loc[idxcommon]

# Assign class labels
Data['label'] = EsInfo['Essential CEG']

# Randomize Data
np.random.seed(69)
Data = Data.sample(frac=1).reset_index()
Data = Data.set_index('genes')

X = Data.iloc[:, :-1]
Y = Data.iloc[:, -1]


              Gene Essential CEG  Essential OEG
0  ENSG00000107581     Essential      Essential
1  ENSG00000068654     Essential            NaN
2  ENSG00000088325     Essential            NaN
4  ENSG00000165732     Essential  Non-essential
5  ENSG00000085840     Essential  Non-essential
Essential CEG
Non-essential    13743
Essential          833
Name: count, dtype: int64
Essential CEG
1    13743
0      833
Name: count, dtype: int64


# Load Bio+CCcfs+embed features
Load the ENS to symbol convertion dictionary for genes 

In [96]:
import os
path = "../../data"
def get_ens_dict(file_path):
    with open(file_path) as f:
        gtf = list(f)

    gtf = [x for x in gtf if not x.startswith('#')]
    gtf = [x for x in gtf if 'gene_id "' in x and 'gene_name "' in x]
    if len(gtf) == 0:
        print('you need to change gene_id " and gene_name " formats')
    
    gtf = list(map(lambda x: (x.split('gene_id "')[1].split('"')[0], x.split('gene_name "')[1].split('"')[0]), gtf))
    gtf = dict(set(gtf))
    return gtf

gtf_dict = get_ens_dict(os.path.join(path,'Homo_sapiens.GRCh38.112.gtf'))

In [97]:
import pandas as pd
import numpy as np
from HELPpy.preprocess.loaders import load_features
path = "../../data"
Data = load_features([os.path.join(path, "Human_Bio.csv"),
                      os.path.join(path, "Human_CCcfs.csv"),
                      os.path.join(path, "Human_EmbN2V_128.csv")], fixnans=[True, True, False], normalizes=['std', 'std', None])
EsInfo = pd.read_csv("Class_labels/Hs.csv", sep=",", header=0)
EsInfo['gene'] = EsInfo.apply(lambda x: gtf_dict[x.Gene] if x.Gene in gtf_dict.keys() else np.nan, axis=1)
EsInfo = EsInfo[EsInfo['gene'].notna()]
EsInfo = EsInfo[EsInfo['Essential CEG'].notna()] # remove rows with NaN in 'Essential CEG'
EsInfo.set_index('gene', inplace=True)
print(EsInfo['Essential CEG'].value_counts())
print(EsInfo.head())

# Generate class labels suitable for Python
EsInfo['Essential CEG'] = EsInfo['Essential CEG'].astype('category').cat.codes
print(EsInfo['Essential CEG'].value_counts())

idxcommon = np.intersect1d(EsInfo.index.values, Data.index.values)
Data = Data.loc[idxcommon]
EsInfo = EsInfo.loc[idxcommon]

# Assign class labels
Data['label'] = EsInfo['Essential CEG']

# Randomize Data
np.random.seed(69)
Data = Data.sample(frac=1).reset_index()
Data = Data.set_index('Gene')

X = Data.iloc[:, :-1]
Y = Data.iloc[:, -1]

Essential CEG
Non-essential    13734
Essential          833
Name: count, dtype: int64
                   Gene Essential CEG  Essential OEG
gene                                                
EIF3A   ENSG00000107581     Essential      Essential
POLR1A  ENSG00000068654     Essential            NaN
TPX2    ENSG00000088325     Essential            NaN
DDX21   ENSG00000165732     Essential  Non-essential
ORC1    ENSG00000085840     Essential  Non-essential
Essential CEG
1    13734
0      833
Name: count, dtype: int64


# Load reduced Hs Features

In [100]:
# Set working directory
#os.chdir("/to/CLEARER_directory/")
EsInfo = pd.read_csv("Class_labels/Hs.csv", sep=",", header=0)
EsInfo = EsInfo[EsInfo['Essential CEG'].notna()] # remove rows with NaN in 'Essential CEG'
print(EsInfo.head())
print(EsInfo['Essential CEG'].value_counts())

# Generate class labels suitable for Python
EsInfo['Essential CEG'] = EsInfo['Essential CEG'].astype('category').cat.codes
EsInfo.set_index('Gene', inplace=True)
print(EsInfo['Essential CEG'].value_counts())

# Load combined features
Data = pd.read_csv("Features/Hs_features_lasso.csv", sep=",", header=0)
Data.set_index('genes', inplace=True)

# reduce to sahred index
idxcommon = np.intersect1d(EsInfo.index.values, Data.index.values)
Data = Data.loc[idxcommon]
EsInfo = EsInfo.loc[idxcommon]

# Assign class labels
Data['label'] = EsInfo['Essential CEG']

# Randomize Data
np.random.seed(69)
Data = Data.sample(frac=1).reset_index()
Data = Data.set_index('genes')

X = Data.iloc[:, :-1]
Y = Data.iloc[:, -1]

              Gene Essential CEG  Essential OEG
0  ENSG00000107581     Essential      Essential
1  ENSG00000068654     Essential            NaN
2  ENSG00000088325     Essential            NaN
4  ENSG00000165732     Essential  Non-essential
5  ENSG00000085840     Essential  Non-essential
Essential CEG
Non-essential    13743
Essential          833
Name: count, dtype: int64
Essential CEG
1    13743
0      833
Name: count, dtype: int64


# Apply to Hs Feature set or Bio+CCcfs+embed features

In [102]:
from HELPpy.models.prediction import VotingEnsembleLGBM, k_fold_cv
clf = VotingEnsembleLGBM(n_voters=11, learning_rate=0.03, n_estimators=190, boosting_type='gbdt', n_jobs=-1, random_state=42)
#clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
df_scores, scores, predictions = k_fold_cv(X, Y, clf, n_splits=5, resample=False, seed=0, show_progress=True, verbose=True)
df_scores

{0: 0, 1: 1}
label
1    13743
0      833
Name: count, dtype: int64



5-fold:   0%|          | 0/5 [00:00<?, ?it/s]

,measure
ROC-AUC,0.9730±0.0020
Accuracy,0.9281±0.0037
BA,0.9094±0.0085
Sensitivity,0.8883±0.0176
Specificity,0.9305±0.0041
MCC,0.5931±0.0137
CM,"[[740, 93], [955, 12788]]"


# Feature selection

In [57]:
selected_feature_indices = np.where(model.get_support())[0]
selected_features = Data.columns[selected_feature_indices] 
coefficients = clf.coef_
print("Selected Features:", selected_features) 
print("Feature Coefficients:", coefficients)
X[selected_features].to_csv("Features/Hs_features_lasso.csv",index=True)
pd.DataFrame(coefficients.ravel()[selected_feature_indices], columns=['lasso_coeff'], 
             index=pd.Series(name='feature', data=Data.columns[selected_feature_indices])).to_csv("Features/Hs_features_lass_coef.csv", index=True)

Selected Features: Index(['A3s', 'Nc', 'Mitochondrion', 'Cell_membrane', 'Endoplasmic_reticulum',
       'Golgi_apparatus', 'Lysosome.Vacuole', 'Peroxisome', 'cgly_count',
       'ngly_posmean',
       ...
       'Pc2.Hydrophilicity.9', 'Pc2.Hydrophobicity.10',
       'Pc2.Hydrophilicity.12', 'Pc2.Hydrophilicity.16',
       'Pc2.Hydrophobicity.21', 'Pc2.Hydrophobicity.26',
       'Pc2.Hydrophilicity.27', 'Pc2.Hydrophilicity.28',
       'Pc2.Hydrophilicity.29', 'Pc2.Hydrophobicity.30'],
      dtype='object', length=4067)
Feature Coefficients: [[ 0.          0.          0.00099024 ... -0.00372794 -0.00397359
   0.        ]]


IndexError: index 2 is out of bounds for axis 0 with size 1

In [75]:
X[selected_features].shape, Data.shape

((14576, 4067), (14576, 41636))

In [43]:
# Feature selection
X_train = X
y_train = Y

# Lasso feature selection using LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, penalty='l1', solver='saga', scoring='roc_auc', max_iter=1000).fit(X_train, y_train)
model = SelectFromModel(clf, prefit=True)
X_train_selected = model.transform(X_train)

# Remove highly correlated features
corr_matrix = pd.DataFrame(X_train_selected).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]
X_train_selected = pd.DataFrame(X_train_selected).drop(to_drop, axis=1)

train_sets[i] = pd.concat([X_train_selected, y_train.reset_index(drop=True)], axis=1)
val_sets[i] = val_sets[i][train_sets[i].columns]

/Users/maurizio/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/maurizio/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/maurizio/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/maurizio/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/maurizio/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/maurizio/miniconda3/lib/python3.10/sit

AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [53]:
# Remove highly correlated features
import numpy as np
np.bool = np.bool_
corr_matrix = pd.DataFrame(X_train_selected).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]
X_train_selected_2 = pd.DataFrame(X_train_selected).drop(to_drop, axis=1)

train_sets[i] = pd.concat([X_train_selected, y_train.reset_index(drop=True)], axis=1)
val_sets[i] = val_sets[i][train_sets[i].columns]

KeyboardInterrupt: 

In [3]:
# Machine learning
def train_rf(train_data):
    X = train_data.iloc[:, :-1]
    y = train_data.iloc[:, -1]
    
    smote = SMOTE()
    X_resampled, y_resampled = smote.fit_resample(X, y)
    
    rf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
    scores = cross_val_score(rf, X_resampled, y_resampled, cv=kf, scoring='roc_auc')
    rf.fit(X_resampled, y_resampled)
    return rf, scores.mean()

results = Parallel(n_jobs=N)(delayed(train_rf)(train_sets[i]) for i in range(N))
rf_list, auc_scores = zip(*results)

# Performance evaluation on test set
def evaluate_model(rf, val_data):
    X_val = val_data.iloc[:, :-1]
    y_val = val_data.iloc[:, -1]
    
    y_pred = rf.predict(X_val)
    y_prob = rf.predict_proba(X_val)[:, 1]
    
    cm = confusion_matrix(y_val, y_pred)
    roc_auc = roc_auc_score(y_val, y_prob)
    precision, recall, _ = precision_recall_curve(y_val, y_prob)
    pr_auc = auc(recall, precision)
    
    return cm, roc_auc, pr_auc

eval_results = [evaluate_model(rf_list[i], val_sets[i]) for i in range(N)]
cm_list, roc_auc_list, pr_auc_list = zip(*eval_results)

metrics = pd.DataFrame({
    'roc_auc': roc_auc_list,
    'pr_auc': pr_auc_list
})

metrics.loc['mean'] = metrics.mean()
metrics.loc['std'] = metrics.std()

metrics.to_csv("test_rf.csv", index=True)

# Performance evaluation on training set
def get_best_kappa(rf):
    results = pd.DataFrame(rf.cv_results_)
    return results.loc[results['mean_test_score'].idxmax()]

train_metrics = pd.concat([get_best_kappa(rf_list[i]) for i in range(5)])
train_metrics.to_csv("train_rf.csv", index=False)